# Flatiron School Mod 3 Project

## Classifying Steam Games by Price 

<h2 style="color:blue"> Overview: </h2>

### What we wanted to find out

We were insterested in seeing if computer games can be separated into 4 separate price categories based on the game information and the how the steam community views the game based on user reviews and recommendation. In this experiment, we tried to classify games as:

1. Less than $\$0.99$
2. Between $\$0.99 \;\&\; \$4.99$
3. Between $\$4.99 \;\&\; \$9.99$
4. More than $\$9.99$

These categories were determined by using the quartiles of the prices from our dataset.

<h2 style="color:blue"> The Experiment </h2>

### Step 1: Import necessary libraries

In [10]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from data_getter import *
import json
import numpy as np
from sklearn.preprocessing import StandardScaler, label_binarize, LabelEncoder
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc, f1_score, mean_squared_error, confusion_matrix, classification_report
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.multiclass import OneVsRestClassifier
from itertools import cycle
from sklearn import svm
from scipy import interp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Step 2: Gather the data

To gather the data we scraped information and reviews of 76,000 games from steam.com. Please take a look at data_getter.py for the functions used to gather and clean the data.

<h4 style="color:green"> Main Data Gathering Functions</h4>

    # retrieve game info for all games
    def retrieve_steam_data(start, end):
        """
        takes a start index and end index as integers to slice the games list above
        return a list of dictionaries of game info
        """
        data_from_steam_ = []
        for num, game in enumerate(games[start: end]):
            if game["appid"] >= 10:
                data_from_steam_.append(get_game_info(game["appid"]))
            # save the data as a json after every thousand iterations
            if num % 1000 == 0 and num >0:
                t = np.random.choice([1,1.1,1.2,1.3,1.4,1.5])
                save_data(data_from_steam_, f"data_{start}_to_{end}.json")
                print(num, "Saved!")
                time.sleep(t)
            # pause making request to the site after every 100 iterations
            elif num % 100 == 0:
                t = np.random.choice([1,1.1,1.2,1.3,1.4,1.5])
                print(num, t)
                time.sleep(t)
        return data_from_steam_ 

    # retrieve user reviews for all games
    def retrieve_steam_reviews(id_lst):
        """
        takes a list of game ids
        return a list of dictionaries of user reviews of all games in the id_list
        """
        data_from_steam_ = []
        for num, appid in enumerate(id_lst):
            data_from_steam_.append({appid: get_reviews(appid)})
            if num % 1000 == 0 and num >0:
                t = np.random.choice([1,1.1,1.2,1.3,1.4,1.5])
                save_data(data_from_steam_, f"upto_{num}_reviews.json")
                print(num, "Saved!")
                time.sleep(t)
            elif num % 100 == 0:
                t = np.random.choice([1,1.1,1.2,1.3,1.4,1.5])
                print(num, t)
                time.sleep(t)
        return data_from_steam_ 


In [ ]:
# uncleaned data of all 76K games
raw_data = open_save_data('76K_raw_data.json')

In [ ]:
len(raw_data)

In [7]:
def save_data(data, file_to_save_to):
        """takes in a filename to save to"""
        with open(file_to_save_to, 'w') as outfile:
            json.dump(data, outfile)
            
save_data(all_76K, "76K_raw_data.json")

In [2]:
cleaned_76K = open_save_data('ALL_cleaned.json')

NameError: name 'open_save_data' is not defined